In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_10k.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_2k.csv
/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_100k.csv
/kaggle/input/playground-series-s5e10/sample_submission.csv
/kaggle/input/playground-series-s5e10/train.csv
/kaggle/input/playground-series-s5e10/test.csv


In [2]:
#!uv pip install -U autogluon > /dev/null
#!uv pip install -U autogluon > /dev/null

In [3]:
#!pip uninstall pyarrow -y
#!pip install pyarrow==17.0.0

In [4]:
#!uv pip install -U autogluon > /dev/null
#!uv pip install autogluon > /dev/null

In [5]:
# ===== CELL 1: Complete Setup with Module Cache Clearing =====
import sys
import subprocess

print("Installing dependencies with uv...")

# Install compatible versions
subprocess.check_call(["uv", "pip", "install", "--quiet", "pyarrow==17.0.0"])
subprocess.check_call(["uv", "pip", "install", "--quiet", "autogluon"])

# CRITICAL: Clear ALL relevant modules from cache
print("Clearing module cache...")
modules_to_clear = []
for module in list(sys.modules.keys()):
    # Clear data science stack modules that might conflict
    if any(pkg in module.lower() for pkg in ['pandas', 'numpy', 'pyarrow', 'ray', 'sklearn', 'scipy']):
        modules_to_clear.append(module)

for module in modules_to_clear:
    del sys.modules[module]

print(f"✓ Cleared {len(modules_to_clear)} modules from cache")
print("✓ Setup complete!")

Installing dependencies with uv...
Clearing module cache...
✓ Cleared 422 modules from cache
✓ Setup complete!


In [6]:
# ===== CELL 2: Imports and Verification =====
import pandas as pd
import pyarrow
import ray
from autogluon.tabular import TabularPredictor

print(f"AutoGluon version: {TabularPredictor.__module__}")
print(f"Ray version: {ray.__version__}")
print(f"PyArrow version: {pyarrow.__version__}")

assert pyarrow.__version__.startswith('17.'), "PyArrow version check failed!"
print("\n✓ All versions verified correctly!")

/usr/local/lib/python3.11/dist-packages/pandas/__init__.py:14: UserWarning: The NumPy module was reloaded (imported a second time). This can in some cases result in small but subtle issues and is discouraged.
  __import__(_dependency)


AutoGluon version: autogluon.tabular.predictor.predictor
Ray version: 2.44.1
PyArrow version: 17.0.0

✓ All versions verified correctly!


In [7]:
from autogluon.tabular import TabularDataset, TabularPredictor
from pathlib import Path
import numpy as np # linear algebra
import pandas as pd 

In [8]:
path = Path('/kaggle/input/playground-series-s5e10/')
path

PosixPath('/kaggle/input/playground-series-s5e10')

In [9]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')
og_df_a = pd.read_csv('/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_100k.csv')

In [10]:
og_df_a

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,rural,2,0.29,70,night,rainy,False,True,evening,False,False,1,0.64
1,highway,1,0.34,25,dim,clear,False,False,morning,False,False,3,0.27
2,rural,2,0.76,70,night,foggy,True,False,evening,True,True,1,0.76
3,rural,3,0.37,70,night,foggy,True,False,morning,False,True,0,0.60
4,highway,3,0.39,45,dim,rainy,False,True,morning,False,False,0,0.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,rural,2,0.61,60,dim,foggy,False,True,evening,False,False,1,0.54
99996,rural,2,0.52,35,dim,foggy,True,True,afternoon,False,False,1,0.18
99997,rural,2,0.08,70,daylight,clear,False,False,evening,True,False,1,0.20
99998,rural,1,0.83,70,dim,foggy,False,True,morning,False,True,0,0.58


In [11]:
train_df

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
id,,,,,,,,,,,,,
0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...
517749,highway,4,0.10,70,daylight,foggy,True,True,afternoon,False,False,2,0.32
517750,rural,4,0.47,35,daylight,rainy,True,True,morning,False,False,1,0.26
517751,urban,4,0.62,25,daylight,foggy,False,False,afternoon,False,True,0,0.19


In [12]:
TabularPredictor.fit??

In [13]:
target = 'accident_risk'
eval_metric = 'rmse'
#train_data = train
train_data = train_df
Time_limit = 3600*10
#Time_limit = 900
problem_type='regression'

In [14]:
%%time
predictor = TabularPredictor(
    label=target, 
    eval_metric=eval_metric,
    verbosity=1,
    problem_type=problem_type
).fit(
    train_data, 
    presets='best_quality',
    excluded_model_types=['KNN'], 
    time_limit=Time_limit,
    #dynamic_stacking=False,  # ← ADD THIS
    #ag_args_fit={'num_gpus': 1}
)

No path specified. Models will be saved in: "AutogluonModels/ag-20251029_144129"
2025-10-29 14:41:31,990	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.ChunkedArray size changed, may indicate binary incompatibility. Expected 64 from C header, got 72 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib._Tabular size changed, may indicate binary incompatibility. Expected 24 from C header, got 32 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Table size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
(_ray_fit pid=3761) /usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
(_ray_fit 

CPU times: user 2h 50min 6s, sys: 58.2 s, total: 2h 51min 4s
Wall time: 10h 11s


In [15]:
%%time
predictor.leaderboard()

CPU times: user 8.86 ms, sys: 999 µs, total: 9.86 ms
Wall time: 9.21 ms


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-0.055889,root_mean_squared_error,517.414136,21737.471170,0.013535,2.277754,3,True,28
1,LightGBM_BAG_L2,-0.055954,root_mean_squared_error,465.858078,17857.678512,5.644407,53.321898,2,True,17
2,ExtraTreesMSE_BAG_L2,-0.055957,root_mean_squared_error,478.743618,18089.436674,18.529947,285.080059,2,True,20
3,NeuralNetFastAI_BAG_L2,-0.055957,root_mean_squared_error,467.579104,18949.863824,7.365433,1145.507209,2,True,21
4,WeightedEnsemble_L2,-0.055965,root_mean_squared_error,42.498163,11290.851087,0.009369,1.125269,2,True,15
5,XGBoost_BAG_L2,-0.055971,root_mean_squared_error,465.771896,17860.273165,5.558225,55.916550,2,True,22
6,CatBoost_BAG_L2,-0.055975,root_mean_squared_error,461.778331,21140.739997,1.564660,3336.383383,2,True,19
7,LightGBMLarge_BAG_L2,-0.055978,root_mean_squared_error,473.700683,17916.138398,13.487012,111.781783,2,True,24
8,CatBoost_r177_BAG_L2,-0.055979,root_mean_squared_error,461.130416,18234.040863,0.916745,429.684248,2,True,25
9,LightGBMLarge_BAG_L1,-0.055996,root_mean_squared_error,20.672365,76.134247,20.672365,76.134247,1,True,9


In [16]:
%%time
autogluon_preds = predictor.predict(test_df)

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.ChunkedArray size changed, may indicate binary incompatibility. Expected 64 from C header, got 72 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib._Tabular size changed, may indicate binary incompatibility. Expected 24 from C header, got 32 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Table size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


CPU times: user 22min 36s, sys: 8.6 s, total: 22min 45s
Wall time: 6min 56s


In [17]:
autogluon_preds

id
517754    0.297216
517755    0.121636
517756    0.183763
517757    0.313939
517758    0.403387
            ...   
690334    0.102492
690335    0.515753
690336    0.248024
690337    0.127750
690338    0.486041
Name: accident_risk, Length: 172585, dtype: float32

In [18]:
!rm submission.csv
sub_df['accident_risk'] = autogluon_preds.values
sub_df.to_csv('submission.csv', index=False)
sub_df.head()

rm: cannot remove 'submission.csv': No such file or directory


,id,accident_risk
0,517754,0.297216
1,517755,0.121636
2,517756,0.183763
3,517757,0.313939
4,517758,0.403387
